<a href="https://colab.research.google.com/github/VivianOuou/NLP-Course/blob/main/course/en/chapter4/section2_Using_pretraained_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using pretrained models (PyTorch)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [8]:
!pip install datasets evaluate transformers[sentencepiece]

In [1]:
from transformers import pipeline

camembert_fill_mask = pipeline("fill-mask", model="camembert-base")
results = camembert_fill_mask("Le camembert est <mask> :)")
results

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing CamembertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

Device set to use cuda:0


[{'score': 0.5239003896713257,
  'token': 7200,
  'token_str': 'délicieux',
  'sequence': 'Le camembert est délicieux :)'},
 {'score': 0.09637771546840668,
  'token': 2183,
  'token_str': 'excellent',
  'sequence': 'Le camembert est excellent :)'},
 {'score': 0.036324575543403625,
  'token': 26202,
  'token_str': 'succulent',
  'sequence': 'Le camembert est succulent :)'},
 {'score': 0.029080064967274666,
  'token': 528,
  'token_str': 'meilleur',
  'sequence': 'Le camembert est meilleur :)'},
 {'score': 0.027218515053391457,
  'token': 1654,
  'token_str': 'parfait',
  'sequence': 'Le camembert est parfait :)'}]

In [5]:
from transformers import CamembertTokenizer, CamembertForMaskedLM

tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
model = CamembertForMaskedLM.from_pretrained("camembert-base")

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing CamembertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
from transformers import CamembertTokenizer, CamembertForMaskedLM
import torch

# 1. 加载模型和分词器
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
model = CamembertForMaskedLM.from_pretrained("camembert-base")

# 2. 输入句子（确保使用模型的正确掩码标记）
text = "Le camembert est <mask> :)"
inputs = tokenizer(text, return_tensors="pt")

# 3. 找到 [MASK] 的位置
mask_token_id = tokenizer.mask_token_id
masked_index = torch.where(inputs["input_ids"][0] == mask_token_id)[0]

# 检查是否找到 [MASK]
if len(masked_index) == 0:
    raise ValueError("未找到 [MASK] token，请检查输入文本或 tokenizer。")

# 4. 模型预测
with torch.no_grad():
    outputs = model(**inputs)
logits = outputs.logits

# 5. 获取 top-k 预测结果
k = 5
mask_logits = logits[0, masked_index, :]
top_tokens = torch.topk(mask_logits, k, dim=-1).indices[0]

# 6. 解码并打印结果
for i, token_id in enumerate(top_tokens):
    predicted_token = tokenizer.decode(token_id)
    print(f"Top {i+1}: {predicted_token}")

# 替换 [MASK] 并输出完整句子
best_prediction = tokenizer.decode(top_tokens[0])
filled_text = text.replace("<mask>", best_prediction)
print(f"\n完整句子: {filled_text}")

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing CamembertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Top 1: délicieux
Top 2: excellent
Top 3: succulent
Top 4: meilleur
Top 5: parfait

完整句子: Le camembert est délicieux :)


In [6]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("camembert-base")
model = AutoModelForMaskedLM.from_pretrained("camembert-base")

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing CamembertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch

# 1. 自动加载模型和分词器
tokenizer = AutoTokenizer.from_pretrained("camembert-base")
model = AutoModelForMaskedLM.from_pretrained("camembert-base")

# 2. 输入句子（使用模型的正确掩码标记）
text = "Le camembert est <mask> :)"
inputs = tokenizer(text, return_tensors="pt")

# 3. 找到 <mask> 的位置
mask_token_id = tokenizer.mask_token_id
masked_index = torch.where(inputs["input_ids"][0] == mask_token_id)[0]

# 检查是否找到 <mask>
if len(masked_index) == 0:
    raise ValueError("未找到 <mask> token，请检查输入文本或 tokenizer。")

# 4. 模型预测
with torch.no_grad():
    outputs = model(**inputs)
logits = outputs.logits  # 形状: [batch_size, seq_len, vocab_size]

# 5. 提取 <mask> 位置的 logits，并取 top-k 预测
k = 5
mask_logits = logits[0, masked_index, :]  # 形状: [1, vocab_size]
top_tokens = torch.topk(mask_logits, k, dim=-1).indices[0]  # 形状: [k]

# 6. 解码并打印结果
print(f"输入句子: {text}")
for i, token_id in enumerate(top_tokens):
    predicted_token = tokenizer.decode(token_id)
    print(f"Top {i+1}: {predicted_token}")

# 7. 替换 <mask> 并输出完整句子
best_prediction = tokenizer.decode(top_tokens[0])
filled_text = text.replace("<mask>", best_prediction)
print(f"\n完整句子: {filled_text}")

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing CamembertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


输入句子: Le camembert est <mask> :)
Top 1: délicieux
Top 2: excellent
Top 3: succulent
Top 4: meilleur
Top 5: parfait

完整句子: Le camembert est délicieux :)


在 Hugging Face `transformers` 库中，使用 CamemBERT 进行**掩码语言建模（Fill-Mask）**主要有三种方法，它们的核心功能相同，但抽象层级和灵活性不同。以下是三种方法的对比和适用场景：

---

### **方法 1：使用 `pipeline`（最高抽象层级）**
```python
from transformers import pipeline

camembert_fill_mask = pipeline("fill-mask", model="camembert-base")
results = camembert_fill_mask("Le camembert est <mask> :)")
```

#### **特点**
1. **极简封装**：  
   - `pipeline` 自动处理所有步骤（分词、模型推理、解码），返回用户友好的结果（包含预测词及其概率）。
   - 输出示例：
     ```python
     [{'score': 0.9, 'token': 1234, 'token_str': 'délicieux', 'sequence': 'Le camembert est délicieux :)'}, ...]
     ```

2. **开箱即用**：  
   - 适合快速实验或生产部署，无需关心模型架构细节。

3. **灵活性低**：  
   - 无法直接修改模型内部逻辑（如调整注意力机制、隐藏层输出等）。

#### **适用场景**
- 快速测试模型效果。
- 简单的下游任务（如完形填空）。

---

### **方法 2：使用 `AutoTokenizer` + `AutoModelForMaskedLM`（中等抽象层级）**
```python
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch

tokenizer = AutoTokenizer.from_pretrained("camembert-base")
model = AutoModelForMaskedLM.from_pretrained("camembert-base")

inputs = tokenizer("Le camembert est <mask> :)", return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)

masked_index = torch.where(inputs["input_ids"][0] == tokenizer.mask_token_id)[0]
top_tokens = torch.topk(outputs.logits[0, masked_index], k=5).indices[0]

for token in top_tokens:
    print(tokenizer.decode(token))
```

#### **特点**
1. **平衡灵活性与易用性**：  
   - 自动选择适合的模型架构（无需手动指定 `CamembertForMaskedLM`）。
   - 允许自定义输入处理、模型输出解析（如调整 `top-k` 的 `k` 值）。

2. **需要手动处理细节**：  
   - 需手动找到 `<mask>` 的位置、解码预测结果。

3. **适合调试**：  
   - 可以访问中间结果（如 `logits`、`hidden_states`）。

#### **适用场景**
- 需要部分自定义逻辑的任务。
- 模型微调前的原型验证。

---

### **方法 3：使用 `CamembertTokenizer` + `CamembertForMaskedLM`（最低抽象层级）**
```python
from transformers import CamembertTokenizer, CamembertForMaskedLM

tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
model = CamembertForMaskedLM.from_pretrained("camembert-base")
# 后续代码与方法2相同
```

#### **特点**
1. **显式指定模型架构**：  
   - 直接调用 `CamembertForMaskedLM`，避免 `AutoModel` 的自动推断。
   - 适用于对模型架构有明确要求的场景（如研究模型内部机制）。

2. **冗余性**：  
   - 功能上与 `AutoModelForMaskedLM` 完全一致，但代码可读性更低（除非需要强调模型类型）。

3. **警告信息**：  
   - 会提示未使用的权重（如 `pooler` 层），这是正常现象。

#### **适用场景**
- 需要明确声明模型类型的场合（如论文代码复现）。
- 对特定架构的底层调试。

---

### **三种方法的核心区别总结**
| 特性                | `pipeline`            | `AutoTokenizer` + `AutoModelForMaskedLM` | `CamembertTokenizer` + `CamembertForMaskedLM` |
|---------------------|-----------------------|------------------------------------------|-----------------------------------------------|
| **抽象层级**         | 最高（全自动）        | 中等（半自动）                           | 最低（手动指定架构）                          |
| **代码复杂度**       | 最简单（1行调用）     | 中等（需处理输入/输出）                  | 最高（需完全手动）                            |
| **灵活性**           | 低                    | 高                                       | 高                                            |
| **访问中间结果**     | 不可                  | 可                                       | 可                                            |
| **适用场景**         | 快速实验、生产部署    | 大多数研究或开发任务                     | 需要明确架构的场景                            |

---

### **如何选择？**
1. **“我只想快速试试模型”** → 用 `pipeline`。
2. **“我要微调或自定义逻辑”** → 用 `AutoTokenizer` + `AutoModelForMaskedLM`。
3. **“我在研究 CamemBERT 的底层实现”** → 用 `CamembertTokenizer` + `CamembertForMaskedLM`。

> 注意：性能上三者无差异，最终调用的都是相同的模型权重。